In [1]:
pip install transformers Pillow

In [2]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import requests

# Load a lightweight BLIP model
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

print("BLIP model loaded successfully!")

# --- Example Usage (Optional) ---
# You can uncomment and run this section to test the model
# url = "https://storage.googleapis.com/sfr-share-research-data/images/img_000000030589.jpg"
# image = Image.open(requests.get(url, stream=True).raw).convert("RGB")

# # Conditional image captioning (e.g., provide a prompt)
# text = "a photography of"
# inputs = processor(image, text, return_tensors="pt")

# out = model.generate(**inputs)
# print(f"Conditional Caption: {processor.decode(out[0], skip_special_tokens=True)}")

# # Unconditional image captioning
# inputs = processor(image, return_tensors="pt")

# out = model.generate(**inputs)
# print(f"Unconditional Caption: {processor.decode(out[0], skip_special_tokens=True)}")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

BLIP model loaded successfully!


In [3]:
import os
from PIL import Image

image_paths = [f for f in os.listdir('/content') if f.endswith(('.jpg', '.jpeg', '.png', '.gif'))]
loaded_images = []

print(f"Found {len(image_paths)} images in /content:")
for img_name in image_paths:
    img_path = os.path.join('/content', img_name)
    try:
        img = Image.open(img_path).convert("RGB")
        loaded_images.append((img_name, img))
        print(f" - Loaded {img_name}")

        # Optional: Generate caption for the loaded image using the BLIP model
        # Uncomment the following lines to use the model
        # inputs = processor(img, return_tensors="pt")
        # out = model.generate(**inputs)
        # caption = processor.decode(out[0], skip_special_tokens=True)
        # print(f"   Caption for {img_name}: {caption}")

    except Exception as e:
        print(f" - Could not load {img_name}: {e}")

print(f"\nSuccessfully loaded {len(loaded_images)} images.")
# loaded_images now contains a list of tuples: (filename, PIL_Image_object)

Found 5 images in /content:
 - Loaded A00360.jpg
 - Loaded A01077.jpg
 - Loaded A00367.jpg
 - Loaded A01072.jpg
 - Loaded A01054.jpg

Successfully loaded 5 images.


In [4]:
# Generate captions and save them in a format suitable for Stable Diffusion training

captioned_images_info = []

# Define a prompt to guide the caption generation for mugshots
# This prompt encourages the model to provide more detail about the person's appearance.
mugshot_prompt = "A detailed description of a person in a mugshot, focusing on their facial features, expression, hair, and any distinguishing characteristics or clothing. The person is pictured from the chest up, with a neutral background, for identification purposes. The subject appears to be"

for img_name, img_pil in loaded_images:
    try:
        # Use conditional captioning with the defined prompt
        inputs = processor(img_pil, mugshot_prompt, return_tensors="pt")

        # Generate caption with increased max_new_tokens to aim for ~100 tokens
        # You can adjust max_new_tokens to get your desired length
        out = model.generate(**inputs, max_new_tokens=100, num_beams=4, early_stopping=True)
        caption = processor.decode(out[0], skip_special_tokens=True)

        # Combine the prompt and the generated caption (optional, depending on training data format)
        # For Stable Diffusion, often just the generated detailed description is used as the caption.
        # If you want the full prompt included, you can concatenate them:
        # full_caption = f"{mugshot_prompt} {caption}"
        # For now, we will use the generated part as the caption

        # Prepare filename for caption file
        base_name = os.path.splitext(img_name)[0]
        caption_filename = f"{base_name}.txt"
        caption_filepath = os.path.join('/content', caption_filename)

        # Save the caption to a text file
        with open(caption_filepath, "w") as f:
            f.write(caption)

        captioned_images_info.append({
            "image_file": img_name,
            "caption_file": caption_filename,
            "caption": caption
        })
        print(f" - Generated caption for '{img_name}' (approx. {len(caption.split())} words): '{caption}' and saved to '{caption_filename}'")

    except Exception as e:
        print(f" - Could not generate caption or save for {img_name}: {e}")

print(f"\nSuccessfully generated and saved {len(captioned_images_info)} detailed captions.")
print("These captions are now ready to be used for training a Stable Diffusion model.")


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

 - Generated caption for 'A00360.jpg' (approx. 42 words): 'a detailed description of a person in a mugshot, focusing on their facial features, expression, hair, and any distinguishing characteristics or clothing. the person is pictured from the chest up, with a neutral background, for identification purposes. the subject appears to be' and saved to 'A00360.txt'
 - Generated caption for 'A01077.jpg' (approx. 42 words): 'a detailed description of a person in a mugshot, focusing on their facial features, expression, hair, and any distinguishing characteristics or clothing. the person is pictured from the chest up, with a neutral background, for identification purposes. the subject appears to be' and saved to 'A01077.txt'
 - Generated caption for 'A00367.jpg' (approx. 42 words): 'a detailed description of a person in a mugshot, focusing on their facial features, expression, hair, and any distinguishing characteristics or clothing. the person is pictured from the chest up, with a neutral ba

In [6]:
pip install diffusers accelerate

In [7]:

from diffusers import StableDiffusionPipeline
import torch

# Load the Stable Diffusion model
# Using a common model like runwayml/stable-diffusion-v1-5
# Ensure you have a GPU runtime enabled for better performance

model_id = "runwayml/stable-diffusion-v1-5"
pipeline = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipeline.to("cuda")

print(f"Stable Diffusion model '{model_id}' loaded successfully!")
print("You can now use 'pipeline' to generate images.")

# --- Example Usage (Optional) ---
# Uncomment the following lines to generate an example image
# prompt = "a photo of an astronaut riding a horse on mars"
# image = pipeline(prompt).images[0]
# image.save("astronaut_horse.png")
# print("Example image 'astronaut_horse.png' generated!")

Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


Stable Diffusion model 'runwayml/stable-diffusion-v1-5' loaded successfully!
You can now use 'pipeline' to generate images.


# Task
Install `bitsandbytes` for memory-efficient training and `xformers` for speed to optimize the LoRA training environment.

## Prepare Training Environment

### Subtask:
Install necessary libraries for LoRA training, such as `bitsandbytes` for memory-efficient training and `xformers` for speed.


**Reasoning**:
The subtask requires installing `bitsandbytes` and `xformers` for LoRA training. I will use `pip install` for both libraries.



In [8]:
pip install bitsandbytes xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 438.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB ? eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 853.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 757.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/

In [2]:
from diffusers import StableDiffusionPipeline
import torch

# Load a lightweight Stable Diffusion model
# stabilityai/stable-diffusion-2-1-base is a good lightweight option.
# runwayml/stable-diffusion-v1-5 (loaded in a previous cell) is also a common lightweight base model.
model_id_light = "runwayml/stable-diffusion-v1-5"
pipeline_light = StableDiffusionPipeline.from_pretrained(model_id_light, torch_dtype=torch.float16)
pipeline_light.to("cuda")

print(f"Lightweight Stable Diffusion model '{model_id_light}' loaded successfully!")
print("You can now use 'pipeline_light' to generate images.")

# Example Usage:
# prompt = "a professional photograph of an astronaut on the moon"
# image = pipeline_light(prompt).images[0]
# image.save("astronaut_moon_light.png")
# print("Example image 'astronaut_moon_light.png' generated!")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


Lightweight Stable Diffusion model 'runwayml/stable-diffusion-v1-5' loaded successfully!
You can now use 'pipeline_light' to generate images.


# Task
Organize the image and caption files into a structured directory named `lora_dataset/images` for LoRA training, ensuring each image has its corresponding caption file co-located.

## Organize Dataset for LoRA

### Subtask:
Create a structured dataset directory, move image files into a subdirectory, and place corresponding caption files next to them, preparing the data for LoRA training.


## Summary:

### Data Analysis Key Findings
- The task involved defining a structured dataset directory, named `lora_dataset/images`, specifically for LoRA model training.
- The defined organization requires moving image files into this `images` subdirectory and placing their corresponding caption files next to each image.

### Insights or Next Steps
- The defined dataset structure is a crucial preparatory step, enabling the subsequent use of this data for LoRA model training.
- The immediate next step is to execute the described file organization to create the `lora_dataset/images` directory with the co-located image and caption files.


# Task
```python
import os
import shutil

# Define the target dataset directory structure
dataset_dir = "lora_dataset"
images_dir = os.path.join(dataset_dir, "images")

# Create the directory structure if it doesn't exist
os.makedirs(images_dir, exist_ok=True)
print(f"Created directory: {images_dir}")

# Iterate through the generated captions and move the image and caption files
# The images and captions are currently in the /content directory
print("\nOrganizing image and caption files:")
for item in captioned_images_info:
    image_name = item["image_file"]
    caption_name = item["caption_file"]

    # Source paths in /content
    src_image_path = os.path.join("/content", image_name)
    src_caption_path = os.path.join("/content", caption_name)

    # Destination paths in lora_dataset/images
    dest_image_path = os.path.join(images_dir, image_name)
    dest_caption_path = os.path.join(images_dir, caption_name)

    try:
        # Move the image file
        shutil.move(src_image_path, dest_image_path)
        print(f"Moved image '{image_name}' to '{dest_image_path}'")

        # Move the caption file
        shutil.move(src_caption_path, dest_caption_path)
        print(f"Moved caption '{caption_name}' to '{dest_caption_path}'")

    except FileNotFoundError:
        print(f"Warning: Image or caption file not found for {image_name}. Skipping.")
    except Exception as e:
        print(f"Error moving files for {image_name}: {e}")

print(f"\nFinished organizing dataset. Images and captions are now in '{images_dir}'.")
```

## Prepare Training Script

### Subtask:
Download or create a Stable Diffusion LoRA training script (e.g., `train_dreambooth_lora.py` from Diffusers examples) that can be used with `accelerate`.


**Reasoning**:
I need to download the `train_dreambooth_lora.py` script from the Hugging Face Diffusers examples GitHub repository. I will use `wget` to directly download the raw file.



In [3]:
import os

# URL of the raw script on GitHub
script_url = "https://raw.githubusercontent.com/huggingface/diffusers/main/examples/text_to_image/train_dreambooth_lora.py"
script_name = "train_dreambooth_lora.py"

# Download the script using wget
!wget -O {script_name} {script_url}

# Verify that the script has been downloaded
if os.path.exists(script_name):
    print(f"Script '{script_name}' downloaded successfully.")
else:
    print(f"Failed to download '{script_name}'.")


--2025-12-14 11:11:33--  https://raw.githubusercontent.com/huggingface/diffusers/main/examples/text_to_image/train_dreambooth_lora.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-12-14 11:11:33 ERROR 404: Not Found.

Script 'train_dreambooth_lora.py' downloaded successfully.


**Reasoning**:
The previous attempt to download the script failed with a 404 error, indicating an incorrect URL. I need to update the `script_url` to the correct path for `train_dreambooth_lora.py` in the Hugging Face Diffusers repository, which is typically found under `examples/dreambooth`. Then, I will retry downloading the script.



In [4]:
import os

# Corrected URL of the raw script on GitHub
script_url = "https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/train_dreambooth_lora.py"
script_name = "train_dreambooth_lora.py"

print(f"Attempting to download script from: {script_url}")

# Download the script using wget
!wget -O {script_name} {script_url}

# Verify that the script has been downloaded
if os.path.exists(script_name):
    print(f"Script '{script_name}' downloaded successfully.")
else:
    print(f"Failed to download '{script_name}'. Please check the URL or your internet connection.")

Attempting to download script from: https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/train_dreambooth_lora.py
--2025-12-14 11:11:49--  https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/train_dreambooth_lora.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59191 (58K) [text/plain]
Saving to: ‘train_dreambooth_lora.py’

train_dreambooth_lo 100%[===================>]  57.80K  --.-KB/s    in 0s      

2025-12-14 11:11:49 (160 MB/s) - ‘train_dreambooth_lora.py’ saved [59191/59191]

Script 'train_dreambooth_lora.py' downloaded successfully.


## Define Training Arguments

### Subtask:
Set up the necessary training arguments, such as the base model, dataset path, LoRA rank, learning rate, batch size, number of training steps, and output directory.


**Reasoning**:
The subtask requires defining several training arguments as Python variables. I will create a code block to define each variable as specified in the instructions, including the model path, dataset path, output directory, prompts, resolution, batch size, gradient accumulation, learning rate, scheduler, warmup steps, number of epochs, and seed.



In [5]:
pretrained_model_name_or_path = "runwayml/stable-diffusion-v1-5"
dataset_name = "lora_dataset"
output_dir = "lora_dreambooth_model"

# Prompts for DreamBooth training
instance_prompt = "a photo of a zyz mugshot person" # 'zyz' is a placeholder for a unique identifier
class_prompt = "a photo of a person"

# Training hyper-parameters
resolution = 512
train_batch_size = 1
gradient_accumulation_steps = 4
learning_rate = 1e-4
lr_scheduler = "constant"
lr_warmup_steps = 0
num_train_epochs = 100
seed = 42

print(f"Pretrained Model: {pretrained_model_name_or_path}")
print(f"Dataset Directory: {dataset_name}")
print(f"Output Directory: {output_dir}")
print(f"Instance Prompt: {instance_prompt}")
print(f"Class Prompt: {class_prompt}")
print(f"Resolution: {resolution}")
print(f"Train Batch Size: {train_batch_size}")
print(f"Gradient Accumulation Steps: {gradient_accumulation_steps}")
print(f"Learning Rate: {learning_rate}")
print(f"LR Scheduler: {lr_scheduler}")
print(f"LR Warmup Steps: {lr_warmup_steps}")
print(f"Number of Training Epochs: {num_train_epochs}")
print(f"Seed: {seed}")

Pretrained Model: runwayml/stable-diffusion-v1-5
Dataset Directory: lora_dataset
Output Directory: lora_dreambooth_model
Instance Prompt: a photo of a zyz mugshot person
Class Prompt: a photo of a person
Resolution: 512
Train Batch Size: 1
Gradient Accumulation Steps: 4
Learning Rate: 0.0001
LR Scheduler: constant
LR Warmup Steps: 0
Number of Training Epochs: 100
Seed: 42


## Summary:

### Training Arguments Key Findings
- **Pretrained Model**: The base Stable Diffusion model `runwayml/stable-diffusion-v1-5` has been selected for fine-tuning.
- **Dataset Path**: The `lora_dataset` directory, where images and captions were organized, is specified as the training data source.
- **Output Directory**: The `lora_dreambooth_model` directory has been designated to store the resulting LoRA model.
- **Prompts**: An `instance_prompt` (`a photo of a zyz mugshot person`) and a `class_prompt` (`a photo of a person`) are defined to guide the DreamBooth training process, enabling the model to learn a specific concept (e.g., 'zyz mugshot person').
- **Hyperparameters**: Key training hyperparameters such as `resolution` (512), `train_batch_size` (1), `gradient_accumulation_steps` (4), `learning_rate` (1e-4), `lr_scheduler` (constant), `lr_warmup_steps` (0), `num_train_epochs` (100), and `seed` (42) have been set. These are crucial for controlling the training process and ensuring reproducibility.

### Insights or Next Steps
- The comprehensive definition of training arguments establishes the foundation for running the LoRA training script.
- The next logical step is to execute the `train_dreambooth_lora.py` script using these defined arguments to begin fine-tuning the Stable Diffusion model.

## Summary:

### Training Arguments Key Findings
- **Pretrained Model**: The base Stable Diffusion model `runwayml/stable-diffusion-v1-5` has been selected for fine-tuning.
- **Dataset Path**: The `lora_dataset` directory, where images and captions were organized, is specified as the training data source.
- **Output Directory**: The `lora_dreambooth_model` directory has been designated to store the resulting LoRA model.
- **Prompts**: An `instance_prompt` (`a photo of a zyz mugshot person`) and a `class_prompt` (`a photo of a person`) are defined to guide the DreamBooth training process, enabling the model to learn a specific concept (e.g., 'zyz mugshot person').
- **Hyperparameters**: Key training hyperparameters such as `resolution` (512), `train_batch_size` (1), `gradient_accumulation_steps` (4), `learning_rate` (1e-4), `lr_scheduler` (constant), `lr_warmup_steps` (0), `num_train_epochs` (100), and `seed` (42) have been set. These are crucial for controlling the training process and ensuring reproducibility.

### Insights or Next Steps
- The comprehensive definition of training arguments establishes the foundation for running the LoRA training script.
- The next logical step is to execute the `train_dreambooth_lora.py` script using these defined arguments to begin fine-tuning the Stable Diffusion model.

## Organize Dataset for LoRA

### Subtask:
Create a structured dataset directory, move image files into a subdirectory, and place corresponding caption files next to them, preparing the data for LoRA training.


In [8]:
import os
import shutil
from PIL import Image

# --- Start: Re-populate loaded_images as it might have been cleared or not run ---
image_paths = [f for f in os.listdir('/content') if f.endswith(('.jpg', '.jpeg', '.png', '.gif'))]
loaded_images = []

print(f"Found {len(image_paths)} images in /content:")
for img_name in image_paths:
    img_path = os.path.join('/content', img_name)
    try:
        img = Image.open(img_path).convert("RGB")
        loaded_images.append((img_name, img))
        print(f" - Re-loaded {img_name}")
    except Exception as e:
        print(f" - Could not re-load {img_name}: {e}")
print(f"\nSuccessfully re-loaded {len(loaded_images)} images.")
# --- End: Re-populate loaded_images ---

# Re-generate captioned_images_info (as it was not defined in the previous execution context)
captioned_images_info = []

mugshot_prompt = "A detailed description of a person in a mugshot, focusing on their facial features, expression, hair, and any distinguishing characteristics or clothing. The person is pictured from the chest up, with a neutral background, for identification purposes. The subject appears to be"

# Assuming 'processor' and 'model' are available from previous cells.
# 'loaded_images' is now guaranteed to be populated.
for img_name, img_pil in loaded_images:
    try:
        inputs = processor(img_pil, mugshot_prompt, return_tensors="pt")
        out = model.generate(**inputs, max_new_tokens=100, num_beams=4, early_stopping=True)
        caption = processor.decode(out[0], skip_special_tokens=True)

        base_name = os.path.splitext(img_name)[0]
        caption_filename = f"{base_name}.txt"
        caption_filepath = os.path.join('/content', caption_filename)

        with open(caption_filepath, "w") as f:
            f.write(caption)

        captioned_images_info.append({
            "image_file": img_name,
            "caption_file": caption_filename,
            "caption": caption
        })
        print(f" - Regenerated caption for '{img_name}' (approx. {len(caption.split())} words).")

    except Exception as e:
        print(f" - Could not regenerate caption for {img_name}: {e}")

print(f"\nSuccessfully regenerated {len(captioned_images_info)} detailed captions.")

# Define the target dataset directory structure
dataset_dir = "lora_dataset"
images_dir = os.path.join(dataset_dir, "images")

# Create the directory structure if it doesn't exist
os.makedirs(images_dir, exist_ok=True)
print(f"\nCreated directory: {images_dir}")

# Iterate through the generated captions and move the image and caption files
# The images and captions are currently in the /content directory
print("\nOrganizing image and caption files:")
for item in captioned_images_info:
    image_name = item["image_file"]
    caption_name = item["caption_file"]

    # Source paths in /content
    src_image_path = os.path.join("/content", image_name)
    src_caption_path = os.path.join("/content", caption_name)

    # Destination paths in lora_dataset/images
    dest_image_path = os.path.join(images_dir, image_name)
    dest_caption_path = os.path.join(images_dir, caption_name)

    try:
        # Move the image file
        shutil.move(src_image_path, dest_image_path)
        print(f"Moved image '{image_name}' to '{dest_image_path}'")

        # Move the caption file
        shutil.move(src_caption_path, dest_caption_path)
        print(f"Moved caption '{caption_name}' to '{dest_caption_path}'")

    except FileNotFoundError:
        print(f"Warning: Image or caption file not found for {image_name}. Skipping.")
    except Exception as e:
        print(f"Error moving files for {image_name}: {e}")

print(f"\nFinished organizing dataset. Images and captions are now in '{images_dir}'.")

Found 0 images in /content:

Successfully re-loaded 0 images.

Successfully regenerated 0 detailed captions.

Created directory: lora_dataset/images

Organizing image and caption files:

Finished organizing dataset. Images and captions are now in 'lora_dataset/images'.


**Reasoning**:
The previous attempts to move files failed because `loaded_images` and `captioned_images_info` were empty, as the files were no longer in the `/content` directory. This suggests the files might have been moved by an earlier, successful execution or partial execution of the file organization logic. To confirm the status of the subtask, I will list the contents of the target directory, `lora_dataset/images`, to see if the image and caption files are already there.



In [9]:
import os

images_dir = "lora_dataset/images"

if os.path.exists(images_dir):
    print(f"Contents of '{images_dir}':")
    files_in_dest = os.listdir(images_dir)
    if files_in_dest:
        for f in files_in_dest:
            print(f" - {f}")
        print(f"\nFound {len(files_in_dest)} files in the target directory. The images and captions are likely already organized.")
    else:
        print(f"The directory '{images_dir}' exists but is empty. Images and captions are missing from both /content and {images_dir}.")
else:
    print(f"The target directory '{images_dir}' does not exist. No files were moved.")

# Based on this check, I will decide if the subtask is complete or if further action is needed.

Contents of 'lora_dataset/images':
 - A01072.txt
 - A00367.txt
 - A01077.txt
 - A00360.jpg
 - A01077.jpg
 - A00367.jpg
 - A01072.jpg
 - A01054.txt
 - A00360.txt
 - A01054.jpg

Found 10 files in the target directory. The images and captions are likely already organized.


## Run LoRA Training

### Subtask:
Execute the LoRA training script using `accelerate launch` with the defined training arguments and the prepared dataset.


**Reasoning**:
The subtask requires executing the LoRA training script using `accelerate launch` with the previously defined training arguments. I will construct the shell command using the Python variables and include recommended flags like `mixed_precision` and `xformers` for optimization, then execute it as a shell command.



In [10]:
import os

# Construct the accelerate launch command
accelerate_command = (
    f"accelerate launch train_dreambooth_lora.py "
    f"--pretrained_model_name_or_path='{pretrained_model_name_or_path}' "
    f"--instance_data_dir='{dataset_name}/images' " # Point to the 'images' subfolder
    f"--output_dir='{output_dir}' "
    f"--instance_prompt='{instance_prompt}' "
    f"--class_prompt='{class_prompt}' "
    f"--resolution={resolution} "
    f"--train_batch_size={train_batch_size} "
    f"--gradient_accumulation_steps={gradient_accumulation_steps} "
    f"--learning_rate={learning_rate} "
    f"--lr_scheduler='{lr_scheduler}' "
    f"--lr_warmup_steps={lr_warmup_steps} "
    f"--num_train_epochs={num_train_epochs} "
    f"--seed={seed} "
    f"--mixed_precision='fp16' " # Use mixed precision for memory efficiency on GPU
    f"--enable_xformers_memory_efficient_attention " # Enable xformers for speed
    f"--checkpointing_steps=500 " # Save checkpoints periodically
    f"--allow_tf32 " # Allow TF32 for faster training on Ampere GPUs
)

print("Executing LoRA training with the following command:")
print(accelerate_command)

# Execute the command
!{accelerate_command}

print("LoRA training command executed. Check the output above for training progress.")

Executing LoRA training with the following command:
accelerate launch train_dreambooth_lora.py --pretrained_model_name_or_path='runwayml/stable-diffusion-v1-5' --instance_data_dir='lora_dataset/images' --output_dir='lora_dreambooth_model' --instance_prompt='a photo of a zyz mugshot person' --class_prompt='a photo of a person' --resolution=512 --train_batch_size=1 --gradient_accumulation_steps=4 --learning_rate=0.0001 --lr_scheduler='constant' --lr_warmup_steps=0 --num_train_epochs=100 --seed=42 --mixed_precision='fp16' --enable_xformers_memory_efficient_attention --checkpointing_steps=500 --allow_tf32 
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-12

**Reasoning**:
The previous training execution failed because the installed `diffusers` version was too old for the script. I need to upgrade the `diffusers` library to meet the minimum version requirement (0.37.0.dev0 or higher) by performing a pip install upgrade. After upgrading, I will re-attempt the `accelerate config` and the training script execution.



In [11]:
print("Upgrading diffusers library...")
!pip install --upgrade diffusers

print("\nRe-running accelerate config...")
# The accelerate config needs to be run in a separate command,
# and it requires user interaction. Since we cannot directly interact
# with it in an automated way, we will try to infer a common configuration
# or instruct the user to run it manually if the script fails again due
# to accelerate configuration issues.
# For now, we will proceed assuming a default GPU configuration if available.

print("\nExecuting LoRA training with the following command:")
accelerate_command = (
    f"accelerate launch train_dreambooth_lora.py "
    f"--pretrained_model_name_or_path='{pretrained_model_name_or_path}' "
    f"--instance_data_dir='{dataset_name}/images' " # Point to the 'images' subfolder
    f"--output_dir='{output_dir}' "
    f"--instance_prompt='{instance_prompt}' "
    f"--class_prompt='{class_prompt}' "
    f"--resolution={resolution} "
    f"--train_batch_size={train_batch_size} "
    f"--gradient_accumulation_steps={gradient_accumulation_steps} "
    f"--learning_rate={learning_rate} "
    f"--lr_scheduler='{lr_scheduler}' "
    f"--lr_warmup_steps={lr_warmup_steps} "
    f"--num_train_epochs={num_train_epochs} "
    f"--seed={seed} "
    f"--mixed_precision='fp16' " # Use mixed precision for memory efficiency on GPU
    f"--enable_xformers_memory_efficient_attention " # Enable xformers for speed
    f"--checkpointing_steps=500 " # Save checkpoints periodically
    f"--allow_tf32 " # Allow TF32 for faster training on Ampere GPUs
)

print(accelerate_command)

# Execute the command
!{accelerate_command}

print("\nLoRA training command executed. Check the output above for training progress.")


Upgrading diffusers library...

Re-running accelerate config...

Executing LoRA training with the following command:
accelerate launch train_dreambooth_lora.py --pretrained_model_name_or_path='runwayml/stable-diffusion-v1-5' --instance_data_dir='lora_dataset/images' --output_dir='lora_dreambooth_model' --instance_prompt='a photo of a zyz mugshot person' --class_prompt='a photo of a person' --resolution=512 --train_batch_size=1 --gradient_accumulation_steps=4 --learning_rate=0.0001 --lr_scheduler='constant' --lr_warmup_steps=0 --num_train_epochs=100 --seed=42 --mixed_precision='fp16' --enable_xformers_memory_efficient_attention --checkpointing_steps=500 --allow_tf32 
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each 

**Reasoning**:
The previous attempt to upgrade `diffusers` did not result in a newer version, as indicated by 'Requirement already satisfied'. To force an actual upgrade and ensure the required version is installed, I will use `pip install --upgrade --force-reinstall diffusers`. Then, I will retry executing the `accelerate launch` command.



In [12]:
print("Upgrading diffusers library with force reinstall...")
!pip install --upgrade --force-reinstall diffusers

print("\nExecuting LoRA training with the following command:")
accelerate_command = (
    f"accelerate launch train_dreambooth_lora.py "
    f"--pretrained_model_name_or_path='{pretrained_model_name_or_path}' "
    f"--instance_data_dir='{dataset_name}/images' " # Point to the 'images' subfolder
    f"--output_dir='{output_dir}' "
    f"--instance_prompt='{instance_prompt}' "
    f"--class_prompt='{class_prompt}' "
    f"--resolution={resolution} "
    f"--train_batch_size={train_batch_size} "
    f"--gradient_accumulation_steps={gradient_accumulation_steps} "
    f"--learning_rate={learning_rate} "
    f"--lr_scheduler='{lr_scheduler}' "
    f"--lr_warmup_steps={lr_warmup_steps} "
    f"--num_train_epochs={num_train_epochs} "
    f"--seed={seed} "
    f"--mixed_precision='fp16' " # Use mixed precision for memory efficiency on GPU
    f"--enable_xformers_memory_efficient_attention " # Enable xformers for speed
    f"--checkpointing_steps=500 " # Save checkpoints periodically
    f"--allow_tf32 " # Allow TF32 for faster training on Ampere GPUs
)

print(accelerate_command)

# Execute the command
!{accelerate_command}

print("\nLoRA training command executed. Check the output above for training progress.")

Upgrading diffusers library with force reinstall...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 118.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.0/521.0 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.5/803.5 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.2/507.2 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.2 MB/s eta 0:00:


Executing LoRA training with the following command:
accelerate launch train_dreambooth_lora.py --pretrained_model_name_or_path='runwayml/stable-diffusion-v1-5' --instance_data_dir='lora_dataset/images' --output_dir='lora_dreambooth_model' --instance_prompt='a photo of a zyz mugshot person' --class_prompt='a photo of a person' --resolution=512 --train_batch_size=1 --gradient_accumulation_steps=4 --learning_rate=0.0001 --lr_scheduler='constant' --lr_warmup_steps=0 --num_train_epochs=100 --seed=42 --mixed_precision='fp16' --enable_xformers_memory_efficient_attention --checkpointing_steps=500 --allow_tf32 
Traceback (most recent call last):
  File "/usr/local/bin/accelerate", line 4, in <module>
    from accelerate.commands.accelerate_cli import main
  File "/usr/local/lib/python3.12/dist-packages/accelerate/commands/accelerate_cli.py", line 19, in <module>
    from accelerate.commands.estimate import estimate_command_parser
  File "/usr/local/lib/python3.12/dist-packages/accelerate/comma

**Reasoning**:
The previous training attempt failed because the installed `diffusers` version was too old and the error message suggested a source install. Also, there was a version conflict with `huggingface-hub` required by `transformers`. To resolve these, I will first uninstall `diffusers`, `transformers`, `accelerate`, and `huggingface-hub` to clear up any conflicting installations. Then, I will install `diffusers` directly from the main branch of its GitHub repository (a source install), which should provide the required development version. Finally, I will reinstall `transformers` and `accelerate` to ensure all libraries are compatible with the newly installed `diffusers` version.



In [ ]:
print("Uninstalling current diffusers, transformers, accelerate, and huggingface-hub...")
!pip uninstall -y diffusers transformers accelerate huggingface-hub

print("\nInstalling diffusers from source (main branch) as required...")
!pip install git+https://github.com/huggingface/diffusers

print("\nRe-installing transformers and accelerate to ensure compatibility...")
!pip install transformers accelerate

print("\nNow attempting to execute LoRA training with the following command:")
accelerate_command = (
    f"accelerate launch train_dreambooth_lora.py "
    f"--pretrained_model_name_or_path='{pretrained_model_name_or_path}' "
    f"--instance_data_dir='{dataset_name}/images' " # Point to the 'images' subfolder
    f"--output_dir='{output_dir}' "
    f"--instance_prompt='{instance_prompt}' "
    f"--class_prompt='{class_prompt}' "
    f"--resolution={resolution} "
    f"--train_batch_size={train_batch_size} "
    f"--gradient_accumulation_steps={gradient_accumulation_steps} "
    f"--learning_rate={learning_rate} "
    f"--lr_scheduler='{lr_scheduler}' "
    f"--lr_warmup_steps={lr_warmup_steps} "
    f"--num_train_epochs={num_train_epochs} "
    f"--seed={seed} "
    f"--mixed_precision='fp16' " # Use mixed precision for memory efficiency on GPU
    f"--enable_xformers_memory_efficient_attention " # Enable xformers for speed
    f"--checkpointing_steps=500 " # Save checkpoints periodically
    f"--allow_tf32 " # Allow TF32 for faster training on Ampere GPUs
)

print(accelerate_command)

# Execute the command
!{accelerate_command}

print("\nLoRA training command executed. Check the output above for training progress.")


Uninstalling current diffusers, transformers, accelerate, and huggingface-hub...
Found existing installation: diffusers 0.36.0
Uninstalling diffusers-0.36.0:
  Successfully uninstalled diffusers-0.36.0
Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
Found existing installation: huggingface_hub 1.2.3
Uninstalling huggingface_hub-1.2.3:
  Successfully uninstalled huggingface_hub-1.2.3

Installing diffusers from source (main branch) as required...
  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-2oz4pqyb
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-2oz4pqyb
  Resolved https://github.com/huggingface/diffusers to commit 17c0e79dbdf53fb6705e9c09cc1a854b84c39249
  Installing build dependencies ... 

In [5]:
from diffusers import StableDiffusionPipeline
import torch
import os

# Re-load the base Stable Diffusion pipeline
model_id = "runwayml/stable-diffusion-v1-5"
pipeline = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipeline.to("cuda")

print("Base Stable Diffusion pipeline loaded.")

# Load the LoRA weights
lora_path = "lora_dreambooth_model/pytorch_lora_weights.safetensors"

# Check if the LoRA file exists
if os.path.exists(lora_path):
    pipeline.load_lora_weights(lora_path)
    print(f"LoRA weights from '{lora_path}' loaded successfully.")
else:
    print(f"Error: LoRA weights file not found at '{lora_path}'. Cannot proceed with inference.")

# Define a list of prompts to generate images
# These prompts should leverage the 'zyz mugshot person' concept we trained for
prompts = [
    "a photo of a zyz mugshot person, professional studio lighting, high detail, sharp focus",
    "a candid shot of a zyz mugshot person smiling, outdoor lighting, natural look",
    "a painting of a zyz mugshot person, impressionist style, vibrant colors",
    "a black and white photo of a zyz mugshot person, serious expression, dramatic shadows",
    "a digital art of a zyz mugshot person, futuristic cyberpunk setting, neon lights"
]

# Create a directory to save the generated images
output_image_dir = "generated_images"
os.makedirs(output_image_dir, exist_ok=True)
print(f"Created output directory: {output_image_dir}")

print("\nGenerating images...")
for i, prompt in enumerate(prompts):
    print(f"Generating image for prompt: '{prompt}'")
    image = pipeline(prompt).images[0]
    image_filename = os.path.join(output_image_dir, f"generated_image_{i+1}.png")
    image.save(image_filename)
    print(f"Saved image to: {image_filename}")

print("\nImage generation complete. Check the 'generated_images' directory for results.")

Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


Base Stable Diffusion pipeline loaded.


No LoRA keys associated to CLIPTextModel found with the prefix='text_encoder'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModel related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new


...

Saved image to: generated_images/generated_image_3.png
Generating image for prompt: 'a black and white photo of a zyz mugshot person, serious expression, dramatic shadows'


  0%|          | 0/50 [00:00<?, ?it/s]

Saved image to: generated_images/generated_image_4.png
Generating image for prompt: 'a digital art of a zyz mugshot person, futuristic cyberpunk setting, neon lights'


  0%|          | 0/50 [00:00<?, ?it/s]

Saved image to: generated_images/generated_image_5.png

Image generation complete. Check the 'generated_images' directory for results.


**Reasoning**:
The previous installation attempt was interrupted, leading to an inconsistent environment. I need to ensure all required libraries (`diffusers` from source, `transformers`, `accelerate`) are correctly installed before attempting to run the training script again. I will repeat the uninstall and install steps to ensure a clean setup.



In [2]:
pretrained_model_name_or_path = "runwayml/stable-diffusion-v1-5"
dataset_name = "lora_dataset"
output_dir = "lora_dreambooth_model"

# Prompts for DreamBooth training
instance_prompt = "a photo of a zyz mugshot person" # 'zyz' is a placeholder for a unique identifier
class_prompt = "a photo of a person"

# Training hyper-parameters
resolution = 512
train_batch_size = 1
gradient_accumulation_steps = 4
learning_rate = 1e-4
lr_scheduler = "constant"
lr_warmup_steps = 0
num_train_epochs = 100
seed = 42

import os

print("Uninstalling current diffusers, transformers, accelerate, and huggingface-hub...")
!pip uninstall -y diffusers transformers accelerate huggingface-hub

print("\nInstalling diffusers from source (main branch) as required...")
!pip install git+https://github.com/huggingface/diffusers

print("\nRe-installing transformers and accelerate to ensure compatibility...")
!pip install transformers accelerate

print("\nNow attempting to execute LoRA training with the following command:")
accelerate_command = (
    f"accelerate launch train_dreambooth_lora.py "
    f"--pretrained_model_name_or_path='{pretrained_model_name_or_path}' "
    f"--instance_data_dir='{dataset_name}/images' " # Point to the 'images' subfolder
    f"--output_dir='{output_dir}' "
    f"--instance_prompt='{instance_prompt}' "
    f"--class_prompt='{class_prompt}' "
    f"--resolution={resolution} "
    f"--train_batch_size={train_batch_size} "
    f"--gradient_accumulation_steps={gradient_accumulation_steps} "
    f"--learning_rate={learning_rate} "
    f"--lr_scheduler='{lr_scheduler}' "
    f"--lr_warmup_steps={lr_warmup_steps} "
    f"--num_train_epochs={num_train_epochs} "
    f"--seed={seed} "
    f"--mixed_precision='fp16' " # Use mixed precision for memory efficiency on GPU
    f"--enable_xformers_memory_efficient_attention " # Enable xformers for speed
    f"--checkpointing_steps=500 " # Save checkpoints periodically
    f"--allow_tf32 " # Allow TF32 for faster training on Ampere GPUs
)

print(accelerate_command)

# Execute the command
!{accelerate_command}

print("\nLoRA training command executed. Check the output above for training progress.")


Uninstalling current diffusers, transformers, accelerate, and huggingface-hub...
Found existing installation: diffusers 0.36.0.dev0
Uninstalling diffusers-0.36.0.dev0:
  Successfully uninstalled diffusers-0.36.0.dev0
Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
Found existing installation: huggingface-hub 0.36.0
Uninstalling huggingface-hub-0.36.0:
  Successfully uninstalled huggingface-hub-0.36.0

Installing diffusers from source (main branch) as required...
  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-u66n5lvl
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-u66n5lvl
  Resolved https://github.com/huggingface/diffusers to commit 17c0e79dbdf53fb6705e9c09cc1a854b84c39249
  Installing build

**Reasoning**:
The previous training attempt failed with a `PIL.UnidentifiedImageError` because the `train_dreambooth_lora.py` script tried to open a text caption file (`.txt`) as an image. This indicates that the `instance_data_dir` passed to the script contained both image and caption files, and the script's `DreamBoothDataset` does not properly filter for image files. To resolve this, I need to restructure the `lora_dataset` by creating a dedicated subdirectory (`lora_dataset/instance_images`) that will *only* contain the image files. The corresponding caption files (`.txt`) will be moved to a separate directory (`lora_dataset/instance_captions`). Then, I will update the `accelerate_command` to point `instance_data_dir` to this new image-only directory.



In [3]:
import os
import shutil

# Define the base dataset directory
dataset_base_dir = "lora_dataset"

# Define the new image-only directory and caption directory
current_mixed_images_dir = os.path.join(dataset_base_dir, "images")
new_instance_images_dir = os.path.join(dataset_base_dir, "instance_images")
new_instance_captions_dir = os.path.join(dataset_base_dir, "instance_captions")

# Create the new directories if they don't exist
os.makedirs(new_instance_images_dir, exist_ok=True)
os.makedirs(new_instance_captions_dir, exist_ok=True)
print(f"Created directory: {new_instance_images_dir}")
print(f"Created directory: {new_instance_captions_dir}")

print("\nSeparating image and caption files...")

# Move files from the old mixed directory to their respective new directories
if os.path.exists(current_mixed_images_dir):
    for filename in os.listdir(current_mixed_images_dir):
        src_path = os.path.join(current_mixed_images_dir, filename)
        if os.path.isfile(src_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.gif')):
                shutil.move(src_path, os.path.join(new_instance_images_dir, filename))
                print(f"Moved image '{filename}' to '{new_instance_images_dir}'")
            elif filename.lower().endswith(('.txt')):
                shutil.move(src_path, os.path.join(new_instance_captions_dir, filename))
                print(f"Moved caption '{filename}' to '{new_instance_captions_dir}'")
    # Optionally, remove the old mixed directory if it's empty after moving files
    if not os.listdir(current_mixed_images_dir):
        os.rmdir(current_mixed_images_dir)
        print(f"Removed empty directory: {current_mixed_images_dir}")

print("\nFile separation complete. Re-attempting LoRA training...")

# Re-define training arguments (ensure they are in scope)
pretrained_model_name_or_path = "runwayml/stable-diffusion-v1-5"
dataset_name = "lora_dataset"
output_dir = "lora_dreambooth_model"
instance_prompt = "a photo of a zyz mugshot person"
class_prompt = "a photo of a person"
resolution = 512
train_batch_size = 1
gradient_accumulation_steps = 4
learning_rate = 1e-4
lr_scheduler = "constant"
lr_warmup_steps = 0
num_train_epochs = 100
seed = 42

# Construct the accelerate launch command with the corrected instance_data_dir
accelerate_command = (
    f"accelerate launch train_dreambooth_lora.py "
    f"--pretrained_model_name_or_path='{pretrained_model_name_or_path}' "
    f"--instance_data_dir='{new_instance_images_dir}' " # Point to the new image-only subfolder
    f"--output_dir='{output_dir}' "
    f"--instance_prompt='{instance_prompt}' "
    f"--class_prompt='{class_prompt}' "
    f"--resolution={resolution} "
    f"--train_batch_size={train_batch_size} "
    f"--gradient_accumulation_steps={gradient_accumulation_steps} "
    f"--learning_rate={learning_rate} "
    f"--lr_scheduler='{lr_scheduler}' "
    f"--lr_warmup_steps={lr_warmup_steps} "
    f"--num_train_epochs={num_train_epochs} "
    f"--seed={seed} "
    f"--mixed_precision='fp16' "
    f"--enable_xformers_memory_efficient_attention "
    f"--checkpointing_steps=500 "
    f"--allow_tf32 "
)

print("Executing LoRA training with the following command:")
print(accelerate_command)

# Execute the command
!{accelerate_command}

print("\nLoRA training command executed. Check the output above for training progress.")

Created directory: lora_dataset/instance_images
Created directory: lora_dataset/instance_captions

Separating image and caption files...
Moved caption 'A01072.txt' to 'lora_dataset/instance_captions'
Moved caption 'A00367.txt' to 'lora_dataset/instance_captions'
Moved caption 'A01077.txt' to 'lora_dataset/instance_captions'
Moved image 'A00360.jpg' to 'lora_dataset/instance_images'
Moved image 'A01077.jpg' to 'lora_dataset/instance_images'
Moved image 'A00367.jpg' to 'lora_dataset/instance_images'
Moved image 'A01072.jpg' to 'lora_dataset/instance_images'
Moved caption 'A01054.txt' to 'lora_dataset/instance_captions'
Moved caption 'A00360.txt' to 'lora_dataset/instance_captions'
Moved image 'A01054.jpg' to 'lora_dataset/instance_images'
Removed empty directory: lora_dataset/images

File separation complete. Re-attempting LoRA training...
Executing LoRA training with the following command:
accelerate launch train_dreambooth_lora.py --pretrained_model_name_or_path='runwayml/stable-diffus

## Save Trained LoRA Model

### Subtask:
Confirm the presence of the saved LoRA model in the specified output directory.


**Reasoning**:
I need to verify the presence of the trained LoRA model file in the specified output directory to confirm the subtask is complete. I will check for the existence of the output directory and then list its contents to find the model file.



In [4]:
import os

# The output_dir variable is defined in a previous cell (be78923c)
# output_dir = "lora_dreambooth_model"

model_file_name = "pytorch_lora_weights.safetensors"
model_path = os.path.join(output_dir, model_file_name)

print(f"Checking for the LoRA model in: {output_dir}")

if os.path.exists(output_dir):
    print(f"Directory '{output_dir}' exists.")
    contents = os.listdir(output_dir)
    if model_file_name in contents:
        print(f"SUCCESS: LoRA model '{model_file_name}' found in '{output_dir}'.")
        print("The LoRA model has been successfully saved.")
    else:
        print(f"WARNING: Model file '{model_file_name}' not found in '{output_dir}'.")
        print(f"Contents of '{output_dir}': {contents}")
else:
    print(f"ERROR: Output directory '{output_dir}' does not exist.")

Checking for the LoRA model in: lora_dreambooth_model
Directory 'lora_dreambooth_model' exists.
SUCCESS: LoRA model 'pytorch_lora_weights.safetensors' found in 'lora_dreambooth_model'.
The LoRA model has been successfully saved.


## Final Task

### Subtask:
Confirm the successful training of the LoRA model and provide instructions on how to use it for inference.


## Summary:

### Data Analysis Key Findings

*   **LoRA Training Script Acquisition**: The `train_dreambooth_lora.py` script was successfully downloaded from the corrected Hugging Face Diffusers repository (from `examples/dreambooth` instead of `examples/text_to_image`) after an initial 404 error.
*   **Training Arguments Defined**: All necessary LoRA training arguments were explicitly set. Key parameters include:
    *   `pretrained_model_name_or_path`: `runwayml/stable-diffusion-v1-5`
    *   `dataset_name`: `lora_dataset`
    *   `output_dir`: `lora_dreambooth_model`
    *   `instance_prompt`: `a photo of a zyz mugshot person`
    *   `class_prompt`: `a photo of a person`
    *   `num_train_epochs`: 100
    *   `learning_rate`: 1e-4
*   **Dataset Organization**: The dataset was successfully organized into a `lora_dataset` directory. Initially, images and caption files were combined in `lora_dataset/images`, which caused a `PIL.UnidentifiedImageError` during training. This was resolved by restructuring the dataset into `lora_dataset/instance_images` for images and `lora_dataset/instance_captions` for text files.
*   **LoRA Model Training**: After resolving multiple dependency conflicts (primarily `diffusers` version incompatibility and issues with `accelerate`, `transformers`, `huggingface-hub` installations) and correcting the dataset structure, the LoRA model training successfully initiated and completed.
*   **Model Saved**: The trained LoRA weights were successfully saved as `pytorch_lora_weights.safetensors` within the specified `lora_dreambooth_model` output directory.

### Insights or Next Steps

*   The successful training and saving of the LoRA model indicate that the fine-tuning process for generating specific concepts (e.g., "zyz mugshot person") based on custom data is complete.
*   The next step involves testing the trained LoRA model by loading it with a Stable Diffusion pipeline and generating new images using the defined `instance_prompt` to confirm its ability to create new content based on the learned concept.
